# 0. Imports

In [8]:
import pandas as pd
import numpy as np

## Helper Functions

In [9]:
def create_descriptive_statistics_table(dataframe, remove_columns = None):
    df_numerical = dataframe.select_dtypes(include=['int64', 'float64'])
    
    if remove_columns != None and isinstance(remove_columns, list):
        df_numerical = df_numerical.drop(remove_columns, axis=1)
        
    df_min = pd.DataFrame(df_numerical.apply(np.min)).T
    df_max = pd.DataFrame(df_numerical.apply(np.max)).T
    df_range = pd.DataFrame(df_numerical.apply(lambda column: column.max() - column.min())).T
    df_mean = pd.DataFrame(df_numerical.apply(np.mean)).T
    df_median = pd.DataFrame(df_numerical.apply(np.median)).T
    df_std = pd.DataFrame(df_numerical.apply(np.std)).T
    df_skew = pd.DataFrame(df_numerical.apply(lambda x: x.skew())).T
    df_kurtosis = pd.DataFrame(df_numerical.apply(lambda x: x.kurtosis())).T
    
    df = pd.concat([df_min, df_max, df_range, df_mean, df_median, df_std, df_skew, df_kurtosis]).T.reset_index()
    df.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    return df

# 1. Projeto de Insights - House Rocket

## 1.1. Questões de Negócio

1. Quais são os imóveis que a House Rocket deveria comprar e por qual preço ?
2. Uma vez a casa comprada, qual o melhor momento para vendê-las e por qual preço ?

## 1.2. Entendimento do Negócio

1. Quais são os imóveis que a House Rocket deveria comprar e por qual preço ?
   - Plano 01:
      - Coletar e limpar os dados.
      - Agrupar os imóveis por região ( zipcode ).
      - Dentro de cada região, será encontrado a mediana do preço do imóvel.
      - Será sugerido os imóveis que estão abaixo do preço mediano da região, que estejam em boas condições e tenham vista para água.
      - Exemplo de Relatório entregue:
            
| Imóvel Cod | Região | Preço do Imóvel | Preço na Mediana | Condições | Status |
| ---------- | ------ | --------------: | ---------------: | --------- | :----: |
| 10330 | Vila Olímpia | R\$ 450.000,00 | R\$ 500.000,00 | 3 | Compra |
| 10335 | Vila Olímpia | R\$ 750.000,00 | R\$ 500.000,00 | 3 | Não Compra |
| 10335 | Vila Olímpia | R\$ 150.000,00 | R\$ 500.000,00 | 1 | Não Compra |

2. Uma vez a casa comprada, qual o melhor momento para vendê-las e por qual preço ?
   - Plano 01:
     - Coletar e limpar os dados (Etapa já cumprida na tarefa anterior).
     - Agrupar os imóveis por região (zipcode) e por sazonalidade (Summer, Inter).
     - Dentro de cada região e sazonalidade, será calculado a mediana do preço.
     - As condições de venda serão:
         1. Se o preço da compra for maior que a mediana da região + sazonalidade. O preço da venda será igual ao preço da compra + 10%
         2. Se o preço da compra for menor que a mediana da região + sazonalidade. O preço da venda será igual ao preço da compra + 30%
     - Exemplo do relatório entregue:


| Imóvel Cod | Região | Temporada | Preço da Mediana | Preço de Compra | Preço de Venda | Lucro | 
| ---------- | :----- | :---------| ---------------: | --------------: | -------------: | ----: |
| 10330 | Vila Olímpia | Verão | R\$ 800.000,00 | R\$ 450.000,00 | R\$ 585.000,00 | R\$ 135.000,00 |
| 10335 | Vila Olímpia | Inverno | R\$ 400.000,00 | R\$ 500.000,00 | R\$ 500.000,00 | R\$ 50.000,00 |

## 1.3. - Coleta dos dados

### 1.3.1. Carregamento dos Dados

In [2]:
df_raw = pd.read_csv('../datasets/kc_house_data.csv')
df_raw.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### 1.3.2. Descrição dos Dados

| Nome da Coluna | Descrição da Coluna |
| :------------- | :------------------ |
| id | ID único de cada imóvel na base de dados |
| date | Data da venda do imóvel |
| price | Preço de venda do imóvel |
| bedrooms | Número de quartos |
| bedrooms | Número de banheiros. Onde 0.5 representa um quarto com vaso sanitário, mas sem chuveiro; 0.75 representa um banheiro que contém uma pia, uma privada, um chuveiro ou uma banheira. Um banheiro completo tradicionalmente possui uma pia, uma privada, um chuveiro e uma banheira, dessa forma, 0.75 representa que o banheiro tem ou um chuveiro ou uma banheira |
| sqft_living | Metragem quadrada (em pés) do espaço interior do imóvel |
| sqft_lot | Metragem quadrada (em pés) do espaço terrestre do imóvel |
| floors | Número de andares do imóvel |
| waterfront | Coluna que representa se o imóvel possui vista para o mar/lago ou não |
| view | Índice de 0 a 4 de quão boa é a vista do imóvel. 0 é a pior vista e 4 é a melhor vista |
| condition | Índice de 1 a 5 sobre a condição (usado) do imóvel. 1 é a pior condição e 5 é a melhor |
| grade	| Índice de 1 a 13 sobre a condição da construção e design do imóvel. 1 é a pior nota e 13 é a melhor nota |
| sqft_above | Metragem quadrada (em pés) do espaço interno do imóvel que está acima do nível do solo |
| sqft_basement | Metragem quadrada (em pés) do espaço interno do imóvel que está abaixo do nível do solo (porão) |
| yr_built | Ano em que o imóvel foi construído |
| yr_renovated | Ano da última reforma do imóvel |
| zipcode | Código postal do imóvel |
| lat | Ponto de latitude do imóvel |
| long | Ponto de longitude do imóvel |
| sqft_living15 | Metragem quadrada (em pés) do espaço habitacional interior para os 15 vizinhos mais próximos |
| sqft_lot15 | A metragem quadrada (em pés) dos terrenos (lotes vazios) dos 15 vizinhos mais próximos |

### 1.3.3. Verificação de NAs e tipos dos Dados

In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

Como não temos dados faltantes, iremos verificar primeiro a tabela de estatística descritiva para verificar as inconsistencias e depois limpar os dados, caso necessário.

Além disso, precisamos arrumar a coluna `date` para o tipo de data.

### 1.3.4. Tabela de Estatística descritiva

In [14]:
df_numerical = create_descriptive_statistics_table(df_raw, ['id'])
df_numerical

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,price,75000.0000,7.700000e+06,7.625000e+06,540088.141767,450000.0000,367118.703181,4.024069,34.585540
1,bedrooms,0.0000,3.300000e+01,3.300000e+01,3.370842,3.0000,0.930040,1.974300,49.063653
2,bathrooms,0.0000,8.000000e+00,8.000000e+00,2.114757,2.2500,0.770145,0.511108,1.279902
3,sqft_living,290.0000,1.354000e+04,1.325000e+04,2079.899736,1910.0000,918.419649,1.471555,5.243093
4,sqft_lot,520.0000,1.651359e+06,1.650839e+06,15106.967566,7618.0000,41419.553273,13.060019,285.077820
5,floors,1.0000,3.500000e+00,2.500000e+00,1.494309,1.5000,0.539976,0.616177,-0.484723
6,waterfront,0.0000,1.000000e+00,1.000000e+00,0.007542,0.0000,0.086515,11.385108,127.632494
7,view,0.0000,4.000000e+00,4.000000e+00,0.234303,0.0000,0.766300,3.395750,10.893022
8,condition,1.0000,5.000000e+00,4.000000e+00,3.409430,3.0000,0.650728,1.032805,0.525764
9,grade,1.0000,1.300000e+01,1.200000e+01,7.656873,7.0000,1.175432,0.771103,1.190932


Como podemos observar, temos imóveis que não possuem nem banheiros e nem quartos (valores mínimos iguais a zero). Na limpeza, esses imóveis serão removidos.

### 1.3.3. Limpeza dos Dados

In [15]:
df1 = df_raw.loc[df_raw['bedrooms'] > 0, :]
df1 = df1.loc[df1['bathrooms'] > 0, :]

#### 1.3.3.1. Recriando a tabela de estatistica descritiva novamente

In [16]:
df_numerical = create_descriptive_statistics_table(df1, ['id'])
df_numerical

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,price,78000.0000,7.700000e+06,7.622000e+06,540202.919757,450000.0000,367125.231146,4.025706,34.604241
1,bedrooms,1.0000,3.300000e+01,3.200000e+01,3.373200,3.0000,0.926277,2.023641,49.821835
2,bathrooms,0.5000,8.000000e+00,7.500000e+00,2.115826,2.2500,0.768966,0.519709,1.279315
3,sqft_living,370.0000,1.354000e+04,1.317000e+04,2080.321850,1910.0000,918.084869,1.473215,5.252102
4,sqft_lot,520.0000,1.651359e+06,1.650839e+06,15099.408760,7618.0000,41411.678105,13.072604,285.495812
5,floors,1.0000,3.500000e+00,2.500000e+00,1.494096,1.5000,0.539670,0.614497,-0.491066
6,waterfront,0.0000,1.000000e+00,1.000000e+00,0.007547,0.0000,0.086547,11.380796,127.534339
7,view,0.0000,4.000000e+00,4.000000e+00,0.234292,0.0000,0.766372,3.396236,10.895665
8,condition,1.0000,5.000000e+00,4.000000e+00,3.409825,3.0000,0.650531,1.036037,0.519237
9,grade,3.0000,1.300000e+01,1.000000e+01,7.657915,7.0000,1.173173,0.788237,1.135148


## 1.4. Exploração dos Dados

1. Descobrir Insights para o time de negócio.
2. Explorar os dados para identificar o impacto dos atributos nos algoritmos de ML.
3. O que são Insights para o negócio?
    - Insights são descobertas, através dos dados, que são inesperada pelas pessoas.
    - Insight precisa ser acionável, caso contrário, ele é apenas uma curiosidade.
    - Exemplo:
        * Durante o período de natal vendê-se mais casas do que na páscoa ( Descoberta )
        * Imóveis com porão são maiores do que imóveis sem porão ( Não-acionável )
        * Imóveis com porão são 40% mais caros do que os outros, na média ( Acionável - Compra imóveis sem porão )
4. Como fazemos para criar as hipóteses ?
    - Toda hipótese de negócio precisa ter 3 características:
        1. Precisa ser uma afirmação.
        2. Precisa fazer uma comparação entre 2 variáveis.
        3. Precisa de um valor base.
        4. Exemplo:
            * H1: Imóveis que possuem vista para água, são 30% mais caros, na média.
            * H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.
            * H3: Imóveis sem porão possuem sqrt_lot, são 50% maiores do que com porão.
            * H4: O crescimento do preço dos imóveis YoY ( Year over Year ) é de 10%
            * H5: Imóveis com 3 banheiros tem um crescimento MoM ( Month over Month ) de 15%